In [75]:
import numpy as np 
import pandas as pd 
from geopy.geocoders import Nominatim
import time
import requests
import json
import geopandas as gpd
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import ast
from sklearn.cluster import DBSCAN
import colorsys
import googlemaps
import time

In [76]:
data = pd.read_csv("df.csv")

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5392 entries, 0 to 5391
Data columns (total 19 columns):
id                  5392 non-null object
RLOIid              2526 non-null object
catchmentName       2526 non-null object
dateOpened          2526 non-null object
datumOffset         294 non-null float64
easting             4758 non-null object
label               5392 non-null object
lat                 4758 non-null object
long                4758 non-null object
northing            4758 non-null object
notation            5391 non-null object
riverName           2526 non-null object
stageScale          2526 non-null object
stationReference    5391 non-null object
status              2526 non-null object
town                4665 non-null object
wiskiID             2511 non-null object
measures            5392 non-null object
station_id          5391 non-null object
dtypes: float64(1), object(18)
memory usage: 800.5+ KB


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
~/anaconda3/envs/cm/lib/python3.6/site-packages/sklearn/cluster/_dbscan.py in fit(
    self=DBSCAN(eps=0.01, min_samples=1),
    X=array([['51.874767', '-1.740083'],
       ['52.8...
       ['51.185918', '0.931329']], dtype=object),
    y=None,
    sample_weight=None
)
    308 
    309         """
--> 310         X = self._validate_data(X, accept_sparse='csr')
        X = array([['51.874767', '-1.740083'],
       ['52.845991', '-0.100848'],
       ['50.976043', '-2.793549'],
       ...,
       ['50.431052', '-4.679467'],
       ['53.324572', '-1.539797'],
       ['51.185918', '0.931329']], dtype=object)
        self._validate_data = <bound method BaseEstimator._validate_data of DBSCAN(eps=0.01, min_samples=1)>
        global accept_sparse = undefined
    311 
    312         if not self.eps > 0.0:

~/anaconda3/envs/cm/lib/python3.6/site

In [ ]:
https://environment.data.gov.uk/flood-monitoring/id/stations/1491TH/readings?_sorted&_limit=196

Data saved successfully.


In [ ]:
https://environment.data.gov.uk/flood-monitoring/id/stations/1491TH/measures

In [ ]:
county_list = {
    
        "Bedfordshire", "Berkshire", "Bristol", "Buckinghamshire", "Cambridgeshire",
        "Cheshire", "City of London", "Cornwall", "Cumbria", "Derbyshire", "Devon",
        "Dorset", "Durham", "East Riding of Yorkshire", "East Sussex", "Essex",
        "Gloucestershire", "Greater London", "Greater Manchester", "Hampshire",
        "Herefordshire", "Hertfordshire", "Isle of Wight", "Kent", "Lancashire",
        "Leicestershire", "Lincolnshire", "Merseyside", "Norfolk", "North Yorkshire",
        "Northamptonshire", "Northumberland", "Nottinghamshire", "Oxfordshire",
        "Rutland", "Shropshire", "Somerset", "South Yorkshire", "Staffordshire",
        "Suffolk", "Surrey", "Tyne and Wear", "Warwickshire", "West Midlands",
        "West Sussex", "West Yorkshire", "Wiltshire", "Worcestershire",
        "Aberdeen City", "Aberdeenshire", "Angus", "Argyll and Bute", "Clackmannanshire",
        "Dumfries and Galloway", "Dundee City", "East Ayrshire", "East Dunbartonshire",
        "East Lothian", "East Renfrewshire", "Edinburgh, City of", "Falkirk", "Fife",
        "Glasgow City", "Highland", "Inverclyde", "Midlothian", "Moray",
        "Na h-Eileanan Siar (Western Isles)", "North Ayrshire", "North Lanarkshire",
        "Orkney Islands", "Perth and Kinross", "Renfrewshire", "Scottish Borders",
        "Shetland Islands", "South Ayrshire", "South Lanarkshire", "Stirling",
        "West Dunbartonshire", "West Lothian",
        "Clwyd", "Dyfed", "Gwent", "Gwynedd", "Mid Glamorgan", "Powys",
        "South Glamorgan", "West Glamorgan",
        "Antrim", "Armagh", "Down", "Fermanagh", "Londonderry", "Tyrone"   
}

{'Greater Manchester', 'Lincolnshire', 'West Sussex', 'Highland', 'Buckinghamshire', 'Kent', 'Nottinghamshire', 'Down', 'Mid Glamorgan', 'Edinburgh, City of', 'Dyfed', 'Wiltshire', 'Somerset', 'Londonderry', 'Na h-Eileanan Siar (Western Isles)', 'Surrey', 'North Lanarkshire', 'Tyrone', 'Essex', 'Moray', 'Durham', 'Cambridgeshire', 'East Dunbartonshire', 'Tyne and Wear', 'Fife', 'Dorset', 'Northumberland', 'West Glamorgan', 'Fermanagh', 'Hertfordshire', 'North Ayrshire', 'Renfrewshire', 'Armagh', 'Devon', 'East Renfrewshire', 'South Glamorgan', 'Shropshire', 'Falkirk', 'City of London', 'Norfolk', 'Stirling', 'Herefordshire', 'Merseyside', 'Aberdeenshire', 'Cheshire', 'Warwickshire', 'Midlothian', 'Argyll and Bute', 'Cornwall', 'Glasgow City', 'Suffolk', 'Rutland', 'Orkney Islands', 'Clwyd', 'North Yorkshire', 'East Ayrshire', 'Leicestershire', 'East Riding of Yorkshire', 'Cumbria', 'Staffordshire', 'Lancashire', 'East Lothian', 'South Lanarkshire', 'Oxfordshire', 'East Sussex', 'Clackm

In [ ]:


def build_upper_left_panel():
    return html.Div(
        id="upper-left",
        className="six columns",
        children=[
            html.H3(
                className="section-title",
                children="Please select the county",
            ),
            html.Div(
                className="control-row-1",
                children=[
                    html.Div(
                        id="state-select-outer",
                        children=[
                            html.Label("Select a region"),
                            dcc.Dropdown(
                                id="state-select",
                                options=[{"label": i, "value": i} for i in county_list],
                                value=county_list[1],
                            ),
                        ],
                    ),
                ],
            ),
            html.Div(
                id="table-container",
                className="table-container",
                children=[
                    html.Div(
                        id="table-upper",
                        children=[
                            html.P("24 hour value chart for the metric"),
                            dcc.Loading(children=html.Div(id="cost-stats-container")),
                        ],
                    ),
                ],
            ),
        ],
    )


In [104]:
data.head(2)

,id,RLOIid,catchmentName,dateOpened,datumOffset,easting,label,lat,long,northing,notation,riverName,stageScale,stationReference,status,town,wiskiID,measures,station_id
0,http://environment.data.gov.uk/flood-monitorin...,7041,Cotswolds,1994-01-01,NaN,417990,Bourton Dickler,51.874767,-1.740083,219610,1029TH,River Dikler,http://environment.data.gov.uk/flood-monitorin...,1029TH,http://environment.data.gov.uk/flood-monitorin...,Little Rissington,1029TH,"[{""@id"": ""http://environment.data.gov.uk/flood...",1029TH
1,http://environment.data.gov.uk/flood-monitorin...,6022,Welland,1992-01-01,2.0,528000,Surfleet Sluice,52.845991,-0.100848,329300,E2043,River Glen,http://environment.data.gov.uk/flood-monitorin...,E2043,http://environment.data.gov.uk/flood-monitorin...,Surfleet Seas End,L31004,"[{""@id"": ""http://environment.data.gov.uk/flood...",E2043


In [106]:
data = pd.read_csv("geocoded_data.csv")

In [108]:
data.head(3)

,id,RLOIid,catchmentName,dateOpened,datumOffset,easting,label,lat,long,northing,notation,riverName,stageScale,stationReference,status,town,wiskiID,measures,station_id,county
0,http://environment.data.gov.uk/flood-monitorin...,7041,Cotswolds,1994-01-01,NaN,417990,Bourton Dickler,51.874767,-1.740083,219610,1029TH,River Dikler,http://environment.data.gov.uk/flood-monitorin...,1029TH,http://environment.data.gov.uk/flood-monitorin...,Little Rissington,1029TH,"[{""@id"": ""http://environment.data.gov.uk/flood...",1029TH,Gloucestershire
1,http://environment.data.gov.uk/flood-monitorin...,6022,Welland,1992-01-01,2.0,528000,Surfleet Sluice,52.845991,-0.100848,329300,E2043,River Glen,http://environment.data.gov.uk/flood-monitorin...,E2043,http://environment.data.gov.uk/flood-monitorin...,Surfleet Seas End,L31004,"[{""@id"": ""http://environment.data.gov.uk/flood...",E2043,Lincolnshire
2,http://environment.data.gov.uk/flood-monitorin...,3072,"Parrett, Brue and West Somerset Streams",1997-01-01,10.0,344383,Gaw Bridge,50.976043,-2.793549,119926,52119,River Parrett,http://environment.data.gov.uk/flood-monitorin...,52119,http://environment.data.gov.uk/flood-monitorin...,Kingsbury Episcopi,520320_FW,"[{""@id"": ""http://environment.data.gov.uk/flood...",52119,East Ayrshire


In [109]:
data.columns

Index(['id', 'RLOIid', 'catchmentName', 'dateOpened', 'datumOffset', 'easting',
       'label', 'lat', 'long', 'northing', 'notation', 'riverName',
       'stageScale', 'stationReference', 'status', 'town', 'wiskiID',
       'measures', 'station_id', 'county'],
      dtype='object')

In [ ]:
app.layout = html.Div(
    className="container scalable",
    children=[
        html.Div(
            id="banner",
            className="banner",
            children=[
                html.Img(src=app.get_asset_url("defra.svg"),className="logo"),
                html.H4("Real time flood monitoring analytics"),

            ],
        ),
        html.Div(
            id="upper-container",
            className="row",
            children=[
                build_upper_left_panel(),
                html.Div(
                    id="geo-map-outer",
                    className="six columns",
                    children=[
                        html.P(
                            id="map-title",
                            children=" {}".format(
                                state_map[state_list[0]]
                            ),
                        ),
                        html.Div(
                            id="geo-map-loading-outer",
                            children=[
                                dcc.Loading(
                                    id="loading",
                                    children=dcc.Graph(
                                        id="geo-map",
                                        figure={
                                            "data": [],
                                            "layout": dict(
                                                plot_bgcolor="#171b26",
                                                paper_bgcolor="#171b26",
                                            ),
                                        },
                                    ),
                                )
                            ],
                        ),
                    ],
                ),
            ],
        ),
        html.Div(
            id="lower-container",
            children=[
                dcc.Graph(
                    id="procedure-plot",
                    figure=generate_procedure_plot(
                        data_dict[state_list[1]], cost_metric[0], init_region, []
                    ),
                )
            ],
        ),
    ],
)


In [117]:
data = pd.read_csv("geocoded_data.csv")

In [120]:
county_list = [
    "Bedfordshire", "Berkshire", "Bristol", "Buckinghamshire", "Cambridgeshire",
    "Cheshire", "City of London", "Cornwall", "Cumbria", "Derbyshire", "Devon",
    "Dorset", "Durham", "East Riding of Yorkshire", "East Sussex", "Essex",
    "Gloucestershire", "Greater London", "Greater Manchester", "Hampshire",
    "Herefordshire", "Hertfordshire", "Isle of Wight", "Kent", "Lancashire",
    "Leicestershire", "Lincolnshire", "Merseyside", "Norfolk", "North Yorkshire",
    "Northamptonshire", "Northumberland", "Nottinghamshire", "Oxfordshire",
    "Rutland", "Shropshire", "Somerset", "South Yorkshire", "Staffordshire",
    "Suffolk", "Surrey", "Tyne and Wear", "Warwickshire", "West Midlands",
    "West Sussex", "West Yorkshire", "Wiltshire", "Worcestershire",
    "Aberdeen City", "Aberdeenshire", "Angus", "Argyll and Bute", "Clackmannanshire",
    "Dumfries and Galloway", "Dundee City", "East Ayrshire", "East Dunbartonshire",
    "East Lothian", "East Renfrewshire", "Edinburgh, City of", "Falkirk", "Fife",
    "Glasgow City", "Highland", "Inverclyde", "Midlothian", "Moray",
    "Na h-Eileanan Siar (Western Isles)", "North Ayrshire", "North Lanarkshire",
    "Orkney Islands", "Perth and Kinross", "Renfrewshire", "Scottish Borders",
    "Shetland Islands", "South Ayrshire", "South Lanarkshire", "Stirling",
    "West Dunbartonshire", "West Lothian",
    "Clwyd", "Dyfed", "Gwent", "Gwynedd", "Mid Glamorgan", "Powys",
    "South Glamorgan", "West Glamorgan",
    "Antrim", "Armagh", "Down", "Fermanagh", "Londonderry", "Tyrone"
]

In [113]:
data.columns

Index(['id', 'RLOIid', 'catchmentName', 'dateOpened', 'datumOffset', 'easting',
       'label', 'lat', 'long', 'northing', 'notation', 'riverName',
       'stageScale', 'stationReference', 'status', 'town', 'wiskiID',
       'measures', 'station_id', 'county'],
      dtype='object')

In [1]:
import pandas as pd
import datetime
import requests
import plotly.express as px

# Calculate 24 hours ago from now in UTC
now_utc = datetime.datetime.utcnow()
since_str = (now_utc - datetime.timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%SZ")

# Construct the URL with parameters
url = f"https://environment.data.gov.uk/flood-monitoring/id/stations/E2043/readings?_sorted&since={since_str}&_limit=1000"

# Make the GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    # Extract readings directly into a DataFrame
    def get_measure_type(measure_url):
        """Extracts the measure type from the measure URL."""
        return measure_url.split('/')[-1].split('-')[2]  # Get the third part after splitting

    readings_list = [
        {
            "dateTime": item["dateTime"],
            "measure_type": get_measure_type(item["measure"]),
            "value": item["value"],
        }
        for item in data.get("items", [])
    ]

    df = pd.DataFrame(readings_list)

    if not df.empty:
        # Convert 'dateTime' column to datetime format
        df["dateTime"] = pd.to_datetime(df["dateTime"])

        # Plot using Plotly
        fig = px.line(
            df,
            x="dateTime",
            y="value",
            color="measure_type",
            markers=True,
            title="Flood Monitoring Station Readings Over Time",
            labels={"value": "Measurement Value", "dateTime": "DateTime"},
        )

        fig.update_layout(
            xaxis_title="DateTime",
            yaxis_title="Measurement Value",
            xaxis_tickangle=-45,
            template="plotly_dark",  # Dark theme for better visualization
            hovermode="x unified",
        )

        fig.show()



In [11]:
pip install nbformat>=4.2.0

Note: you may need to restart the kernel to use updated packages.


In [38]:
data["label"] = data["label"].astype(str)

In [56]:
df.columns

Index(['id', 'RLOIid', 'catchmentName', 'dateOpened', 'datumOffset', 'easting',
       'label', 'lat', 'long', 'northing', 'notation', 'riverName',
       'stageScale', 'stationReference', 'status', 'town', 'wiskiID',
       'measures', 'station_id', 'county'],
      dtype='object')

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
from dash_table import DataTable
import colorsys
import geopandas as gpd
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN

# Load data
data = pd.read_csv("geocoded_data.csv")

app = dash.Dash(__name__)

def build_upper_left_panel():
    return html.Div(
        id="upper-left",
        className="six columns",
        children=[
            html.H3(
                className="section-title",
                children="Please select the station",
                style={"color": "#2E8B57"}
            ),
            html.Div(
                className="control-row-1",
                children=[
                    html.Div(
                        id="county-select-outer",
                        children=[
                            html.Label(
                                "choose from the dropdown below",
                                style={"color": "#FF4500", "font-weight": "bold"}
                            ),
                           dcc.Dropdown(
                               id="station-select",
                               options=[
                                   {"label": str(station), "value": str(station)}
                                   for station in data["station_id"].unique() if pd.notna(station)
                               ],
                               value=str(data["station_id"].unique()[0]),
                               style={
                                   "backgroundColor": "green",
                                   "color": "green",
                                   "border": "2px solid #FFD700",
                                   "borderRadius": "5px",
                                   "fontFamily": "Arial",
                                   "fontSize": "16px",
                               },
                           )
                        ],
                    ),
                ],
            ),
        ],
    )

def generate_colors(n):
    colors = []
    for i in range(n):
        hue = i / n
        rgb = colorsys.hsv_to_rgb(hue, 0.7, 0.9)
        hex_color = '#{:02x}{:02x}{:02x}'.format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))
        colors.append(hex_color)
    return colors

def generate_geo_map(plot_data):
    geojson_file = 'uk.geojson'
    gdf = gpd.read_file(geojson_file)
    center_lat = gdf.geometry.centroid.y.mean()
    center_lon = gdf.geometry.centroid.x.mean()
    
    scatter_data = go.Scattermapbox(
        lat=plot_data['lat'],
        lon=plot_data['long'],
        mode='markers',
        marker=dict(
            size=8,
            color=plot_data['point_color'],
            opacity=plot_data['opacity'],
            showscale=False
        ),
        text=plot_data.apply(
            lambda row: f"Town: {row['town']}<br>"
                        f"Label: {row['label']}<br>"
                        f"Station ID: {row['station_id']}<br>"
                        f"Notation: {row['notation']}<br>"
                        f"Lat: {row['lat']:.4f}<br>"
                        f"Long: {row['long']:.4f}",
            axis=1
        ),
        hoverinfo='text',
        customdata=plot_data['station_id']
    )
    
     layout = go.Layout(
         margin=dict(l=20, r=20, t=40, b=20, pad=10),
         plot_bgcolor="#121212",  # Darker shade for better contrast
         paper_bgcolor="#121212",
         clickmode="event+select",
         hovermode="closest",
         showlegend=False,
         font=dict(
             color="white",
             family="Arial, sans-serif",
             size=14
         ),
         mapbox=go.layout.Mapbox(
             center=go.layout.mapbox.Center(
                 lat=center_lat,
                 lon=center_lon
             ),
             zoom=5,  # Slightly increased zoom for better details
             style="carto-darkmatter"
         ),
         title=dict(
             text="Enhanced Map Visualization",
             font=dict(size=18, color="white"),
             x=0.5,  # Centered title
             y=0.95
         )
     )
     
    
    return {"data": [scatter_data], "layout": layout}

app.layout = html.Div([
    build_upper_left_panel(),
    html.Div(id="station-stats-table"),
    dcc.Graph(id='geo-map')
])

@app.callback(
    [Output("geo-map", "figure"),
     Output("station-select", "value"),
     Output("station-stats-table", "children")],
    [Input("geo-map", "clickData"),
     Input("station-select", "value")]
)
def update_geo_map_and_ui(click_data, id_select):
    data_modified = data.copy()

    def convert_coord(val, index):
        if isinstance(val, str) and ',' in val:
            parts = val.strip("[]").split(",")
            return float(parts[index].strip())
        else:
            return float(val)

    data_modified['lat'] = data_modified['lat'].apply(lambda x: convert_coord(x, 0))
    data_modified['long'] = data_modified['long'].apply(lambda x: convert_coord(x, 1))

    # Clustering
    X = data_modified[['lat', 'long']].to_numpy()
    clustering = DBSCAN(eps=0.01, min_samples=1).fit(X)
    data_modified['cluster'] = clustering.labels_
    unique_clusters = data_modified['cluster'].unique()
    cluster_colors = generate_colors(len(unique_clusters))
    color_mapping = dict(zip(unique_clusters, cluster_colors))
    data_modified['point_color'] = data_modified['cluster'].map(color_mapping)
    data_modified['opacity'] = 1.0

    # Determine the selected station: prioritize map click if available
    if click_data is not None:
        clicked_station = click_data['points'][0]['customdata']
    else:
        clicked_station = id_select

    data_modified.loc[data_modified['station_id'] != clicked_station, 'opacity'] = 0.2
    data_modified.loc[data_modified['station_id'] == clicked_station, 'point_color'] = 'black'

    # Update the table with the selected station's details
    station_df = data[data["station_id"] == clicked_station]
    selected_columns = ["notation", "riverName", "town", "label", "station_id"]
    filtered_df = station_df[selected_columns]

    table_data = filtered_df.to_dict("records")
    table_columns = [{"name": col, "id": col} for col in selected_columns]

    table_component = html.Div(
DataTable(
    id="station-stats-table-table",
    columns=table_columns,
    data=table_data,
    filter_action="native",
    page_size=5,
    style_cell={
        "background-color": "#2d3e32",  # Dark green-gray background
        "color": "#e0f2e9",  # Light green text
        "textAlign": "center",
        "border": "1px solid #4a7c59",
    },
    style_header={
        "background-color": "#3a5a40",  # Deep green header
        "color": "#ffffff",  # White text for contrast
        "fontWeight": "bold",
        "textAlign": "center",
        "padding": "5px",
    },
    style_data_conditional=[
        {"if": {"row_index": "odd"}, "backgroundColor": "#4f6f52"},  # Alternating row colors
        {"if": {"row_index": "even"}, "backgroundColor": "#2d3e32"},
        {"if": {"state": "selected"}, "backgroundColor": "#81c784", "color": "#000"},  # Highlight selection
        {
            "if": {"column_id": "status"},
            "backgroundColor": "#66bb6a",  # Bright green for emphasis
            "color": "white",
            "fontWeight": "bold",
        },  
    ],
)

    )

    return generate_geo_map(data_modified), clicked_station, table_component

if __name__ == '__main__':
    app.run_server(debug=True)


IndentationError: unexpected indent (<ipython-input-3-43684d8e56e0>, line 127)